- 学修番号: 22745102
- 氏名: 岩間和也

# 課題 1.1　線形回帰分析

In [1]:
import pandas as pd

autos = pd.read_csv('data/imports-85.data', na_values='?')
autos.columns = ['symboling', 'normalized_losses', 'make', 'fuel_type', 'aspiration',
                    'num_of_doors', 'body_style', 'drive_wheels', 'engine_location',
                    'wheel_base', 'length','width', 'height', 'curb_weight',
                    'engine_type', 'num_of_cylinders', 'engine_size', 'fuel_system',
                    'bore', 'stroke', 'compression_ratio', 'horsepower', 'peak_rpm',
                    'city_mpg', 'highway_mpg','price']
# 表示する最大列数の設定
pd.set_option('display.max_columns', len(autos.columns))
# 欠損値 NaN がある行の除去
autos = autos.dropna()
# 確認
autos

,symboling,normalized_losses,make,fuel_type,aspiration,num_of_doors,body_style,drive_wheels,engine_location,wheel_base,length,width,height,curb_weight,engine_type,num_of_cylinders,engine_size,fuel_system,bore,stroke,compression_ratio,horsepower,peak_rpm,city_mpg,highway_mpg,price
2,2,164.0,audi,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,54.3,2337,ohc,four,109,mpfi,3.19,3.40,10.0,102.0,5500.0,24,30,13950.0
3,2,164.0,audi,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,54.3,2824,ohc,five,136,mpfi,3.19,3.40,8.0,115.0,5500.0,18,22,17450.0
5,1,158.0,audi,gas,std,four,sedan,fwd,front,105.8,192.7,71.4,55.7,2844,ohc,five,136,mpfi,3.19,3.40,8.5,110.0,5500.0,19,25,17710.0
7,1,158.0,audi,gas,turbo,four,sedan,fwd,front,105.8,192.7,71.4,55.9,3086,ohc,five,131,mpfi,3.13,3.40,8.3,140.0,5500.0,17,20,23875.0
9,2,192.0,bmw,gas,std,two,sedan,rwd,front,101.2,176.8,64.8,54.3,2395,ohc,four,108,mpfi,3.50,2.80,8.8,101.0,5800.0,23,29,16430.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,-1,95.0,volvo,gas,std,four,sedan,rwd,front,109.1,188.8,68.9,55.5,2952,ohc,four,141,mpfi,3.78,3.15,9.5,114.0,5400.0,23,28,16845.0
200,-1,95.0,volvo,gas,turbo,four,sedan,rwd,front,109.1,188.8,68.8,55.5,3049,ohc,four,141,mpfi,3.78,3.15,8.7,160.0,5300.0,19,25,19045.0
201,-1,95.0,volvo,gas,std,four,sedan,rwd,front,109.1,188.8,68.9,55.5,3012,ohcv,six,173,mpfi,3.58,2.87,8.8,134.0,5500.0,18,23,21485.0
202,-1,95.0,volvo,diesel,turbo,four,sedan,rwd,front,109.1,188.8,68.9,55.5,3217,ohc,six,145,idi,3.01,3.40,23.0,106.0,4800.0,26,27,22470.0


In [2]:
# price 以外の量的データのラベルを昇順にソート
qd_labels = autos.describe().columns.drop('price').sort_values()
# 確認
qd_labels

Index(['bore', 'city_mpg', 'compression_ratio', 'curb_weight', 'engine_size',
       'height', 'highway_mpg', 'horsepower', 'length', 'normalized_losses',
       'peak_rpm', 'stroke', 'symboling', 'wheel_base', 'width'],
      dtype='object')

<hr>

### 1.1.1 VIFが5以上の説明変数を除外

price 以外の量的データである説明変数について、昇順にソートしたラベル qd_labels の先頭からVIFが5以上の変数を除外し、残った説明変数について、説明変数のVIFを行とする pandas DataFrame を作成し、表示しなさい。
- 結果は、行ラベルは説明変数、列ラベルはVIFのn行1列の DataFrame

<hr>

### 方針

- VIF = 1 / (1 - r2_score)
- 1 / (1 - r2_score) >= 5 ということは
- 0.2 >= 1 - r2_score
- さらに式変形すると r2_score >= 0.8
- つまり r2_scoreが 0.8以上の変数を除外すればよい

In [3]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# 該当する変数のラベルのみから構成される配列をつくる
target_label_arr = []
target_val_arr = []
count = 1


quantitive_data_autos = autos.loc[:, qd_labels]


for label in qd_labels:
    label_str = f'{label}'
    remain_labels = qd_labels.drop(label_str)
    count+= 1
    model = LinearRegression()
    
    # 各量的データのラベルを説明変数に、それ以外を目的変数にして線形回帰モデルを作成
    X = quantitive_data_autos[[label_str]]
    Y = quantitive_data_autos.drop([label_str], axis=1)
    model.fit(X, Y)
    Y_predict = model.predict(X)
    
    # r2_scoreが0.8より小さいもののみの配列をラベルと値で作成する
    if r2_score(Y, Y_predict) < 0.8:
        target_label_arr.append(label_str)
        target_val_arr.append(r2_score(Y, Y_predict))

# この結果をデータフレーム化する
df = pd.DataFrame(
    data = {
        'VIF': pd.Series(target_val_arr, index=target_label_arr)
    }
)

# 解答を表示
df

,VIF
bore,0.221666
city_mpg,0.325333
compression_ratio,0.052310
curb_weight,0.397907
engine_size,0.329767
height,0.104871
highway_mpg,0.332271
horsepower,0.307672
length,0.366795
normalized_losses,0.054921


<hr>

### 1.1.2 決定係数

すべての質的データの説明変数と1.1.1で残った説明変数を合わせて説明変数、price を目的変数として線形重回帰モデルを作成し、そのモデルの決定係数を求めなさい。<br>
小数点以下4桁目を四捨五入し、小数点以下3桁まで答えること（0.7625ならば0.763が答え）。

In [4]:
import numpy as np

# qd_labelsとpriceが量的データなので削除対象として配列にする
drop_target_labels = np.append(qd_labels.values, 'price')

# 質的データラベルのみを取り出す
qualititive_labels = autos.columns.drop(labels=drop_target_labels)

# 1.1.1のラベルと結合
merged_labels = np.append(target_label_arr, qualititive_labels)
merged_labels

# モデル作成
_data = autos[merged_labels]
X = pd.get_dummies(data=_data, drop_first=True) # 質的データのダミー変数化
Y = autos.price
model = LinearRegression()
model.fit(X, Y)

# 決定係数をもとめる
Y_predict = model.predict(X)
score = r2_score(Y, Y_predict)

# 解答の値を表示
score

0.9734025468478159

<hr>

### score = 0.9734025468478159
### よって決定係数は0.973

<hr>

### 1.1.3 回帰係数

1.1.2で作成したモデルの回帰係数を、説明変数と回帰係数を行とする pandas DataFrame を作成し、降順で表示しなさい（行数が多い場合、途中の表示が省略されないようにすること）。

<hr>

### 方針

- model.coef_　
- ここには各説明変数の回帰係数が配列として格納されている
- これを順番に取り出して[説明変数, 回帰係数]の形で別の配列に詰め直す
- 詰め直した配列をデータフレーム化する

In [5]:
#     説明変数　回帰係数
# 行１
# 行2
# 行3
# ...
# という形式

data_array = []

for i, label in enumerate(X.columns):
    # ラベルに対する回帰係数を求めてdata_arrayに格納する
    data_array.append([label, model.coef_[i]])
    
# 回帰係数の降順で表示
df2 = pd.DataFrame(data_array, columns=['説明変数', '回帰係数']).sort_values('回帰係数', ascending=False)

# 解答の表示
df2

,説明変数,回帰係数
50,fuel_system_idi,5323.372967
26,make_porsche,4830.408327
51,fuel_system_mfi,3467.897329
52,fuel_system_mpfi,2601.780297
21,make_mercedes-benz,2548.338064
19,make_jaguar,2430.797572
33,aspiration_turbo,2171.490389
49,fuel_system_2bbl,2069.509402
40,drive_wheels_rwd,1977.187880
53,fuel_system_spdi,1080.926758
